In [31]:
#!/usr/bin/env python
# coding: utf-8

import cv2
import numpy as np
import matplotlib.pyplot as plt
import imageio
import imutils
import scipy
from scipy import signal
cv2.ocl.setUseOpenCL(False)

# select the image id (valid values 1,2,3, or 4)
feature_extractor = 'brisk' # one of 'sift', 'surf', 'brisk', 'orb'
feature_matching = 'bf'

def detectAndDescribe(image, method=None):
    """
    Compute key points and feature descriptors using an specific method
    """
    
    assert method is not None, "You need to define a feature detection method. Values are: 'sift', 'surf'"
    
    # detect and extract features from the image
    if method == 'sift':
        descriptor = cv2.xfeatures2d.SIFT_create()
    elif method == 'surf':
        descriptor = cv2.xfeatures2d.SURF_create()
    elif method == 'brisk':
        descriptor = cv2.BRISK_create()
    elif method == 'orb':
        descriptor = cv2.ORB_create()
        
    # get keypoints and descriptors
    (kps, features) = descriptor.detectAndCompute(image, None)
    
    return (kps, features)

def createMatcher(method,crossCheck):
    "Create and return a Matcher Object"
    
    if method == 'sift' or method == 'surf':
        bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=crossCheck)
    elif method == 'orb' or method == 'brisk':
        bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=crossCheck)
    return bf

def matchKeyPointsBF(featuresA, featuresB, method):
    bf = createMatcher(method, crossCheck=True)
        
    # Match descriptors.
    best_matches = bf.match(featuresA,featuresB)
    
    # Sort the features in order of distance.
    # The points with small distance (more similarity) are ordered first in the vector
    rawMatches = sorted(best_matches, key = lambda x:x.distance)
    print("Raw matches (Brute force):", len(rawMatches))
    return rawMatches

def matchKeyPointsKNN(featuresA, featuresB, ratio, method):
    bf = createMatcher(method, crossCheck=False)
    # compute the raw matches and initialize the list of actual matches
    rawMatches = bf.knnMatch(featuresA, featuresB, 2)
    print("Raw matches (knn):", len(rawMatches))
    matches = []

    # loop over the raw matches
    for m,n in rawMatches:
        # ensure the distance is within a certain ratio of each
        # other (i.e. Lowe's ratio test)
        if m.distance < n.distance * ratio:
            matches.append(m)
    return matches

def getHomography(kpsA, kpsB, featuresA, featuresB, matches, reprojThresh):
    # convert the keypoints to numpy arrays
    kpsA = np.float32([kp.pt for kp in kpsA])
    kpsB = np.float32([kp.pt for kp in kpsB])
    
    if len(matches) > 4:

        # construct the two sets of points
        ptsA = np.float32([kpsA[m.queryIdx] for m in matches])
        ptsB = np.float32([kpsB[m.trainIdx] for m in matches])
        
        # estimate the homography between the sets of points
        #(H, status) = cv2.findHomography(ptsA, ptsB, cv2.RANSAC,reprojThresh)
        (H, status) = cv2.estimateAffine2D(ptsA,ptsB,True)
        H = np.vstack((H,[[0,0,1]]))
        return (matches, H, status)
    else:
        return None


def warpTwoImages(img1, img2, H):
    '''warp img2 to img1 with homograph H'''
    h1,w1 = img1.shape[:2]
    h2,w2 = img2.shape[:2]
    pts1 = np.array([[0,0],[0,h1],[w1,h1],[w1,0]],dtype="float32").reshape(-1,1,2)
    pts2 = np.array([[0,0],[0,h2],[w2,h2],[w2,0]],dtype="float32").reshape(-1,1,2)
    
    print("Shape of:",np.shape(pts1))
    pts2_ = cv2.perspectiveTransform(pts2, H)
    #pts2_ = cv2.transform(pts2,H)
    pts = np.concatenate((pts1, pts2_), axis=0)
    [xmin, ymin] = np.int32(pts.min(axis=0).ravel() - 0.5)
    [xmax, ymax] = np.int32(pts.max(axis=0).ravel() + 0.5)
    t = [-xmin,-ymin]
    #Ht = np.array([[1,0,t[0]],[0,1,t[1]],[0,0,1]]) # translate
    Ht = np.array([[1,0,t[0]],[0,1,t[1]],[0,0,1]])
    result = cv2.warpPerspective(img2, Ht.dot(H), (xmax-xmin, ymax-ymin),borderMode=cv2.BORDER_CONSTANT,borderValue=(255, 255, 255))
    result[t[1]:h1+t[1],t[0]:w1+t[0]] =  img1
    return result

#Stich((img1)trainImg_gray, (img2)queryImg_gray,img1c, img2c,feature_extractor):
def Stich(img1, img2, img1c, img2c,feature_extractor):
    kpsA, featuresA = detectAndDescribe(img1, method=feature_extractor)
    kpsB, featuresB = detectAndDescribe(img2, method=feature_extractor)
    
    if feature_matching == 'bf':
        matches = matchKeyPointsBF(featuresA, featuresB, method=feature_extractor)
        img3 = cv2.drawMatches(img1,kpsA,img2,kpsB,matches[:100],
                               None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    elif feature_matching == 'knn':
        matches = matchKeyPointsKNN(featuresA, featuresB, ratio=0.75, method=feature_extractor)
        img3 = cv2.drawMatches(img1,kpsA,img2,kpsB,np.random.choice(matches,100),
                               None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
        
    M = getHomography(kpsA, kpsB, featuresA, featuresB, matches, reprojThresh=4)
    if M is None:
        print("Error!")
    (matches, H, status) = M
    
    #result = warpTwoImages(queryImg(img2c), trainImg(img1c), H)
    result = warpTwoImages(img2c, img1c, H)
    
    gray = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY)[1]
    
    # Finds contours from the binary image
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    
    # get the maximum contour area
    c = max(cnts, key=cv2.contourArea)
    
    # get a bbox from the contour area
    (x, y, w, h) = cv2.boundingRect(c)
    # crop the image to the bbox coordinates
    crop_pad = 0
    result = result[y+crop_pad:y + h-crop_pad, x+crop_pad:x + w-crop_pad]
    sva,svb,svc = np.linalg.svd(H[0:2,0:2])
    s_ratio = svb[0]/svb[-1]
    return result, s_ratio

def AutoCrop(im):
    img = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
    m1 = cv2.Laplacian(img,cv2.CV_64F)
    m1 = np.abs(m1)
    p = np.var(m1,axis = 1)
    p = scipy.signal.medfilt(p,11)
    p[p<0.15]=0
    
    if len(np.nonzero(p)[0])==0:
        return im
    else:
        l = np.max(np.nonzero(p)[0])
        return im[0:l+1,:,:]


def AutoCropLen(im):
    img = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
    m1 = cv2.Laplacian(img,cv2.CV_64F)
    m1 = np.abs(m1)
    p = np.var(m1,axis = 1)
    p = scipy.signal.medfilt(p,11)
    p[p<0.15]=0
    if len(np.nonzero(p)[0])==0:
        l = np.shape(img)[0]
    else:
        l = np.max(np.nonzero(p)[0])
    return l

def FrameSelect(im_list):
    im_list = np.array(im_list)
    level = 1 
    no_of_images = len(im_list)
    effective_len = np.zeros(no_of_images)
    for i in range(0,no_of_images):
        # Read files
        fname1 = path + 'L'+str(level-1).zfill(2) + '_' + str(im_list[i]).zfill(5) + '.jpg'
        ## AutoCrop to Eliminate White Spaces
        trainImg = imageio.imread(fname1)
        effective_len[i] = AutoCropLen(trainImg)
    
    d_len = np.diff(effective_len)
    d_len[d_len>-5]=0
    s_im_list = im_list[np.nonzero(d_len)[0]]
    s_im_list = np.concatenate((s_im_list,[im_list[-1]]))
    return s_im_list.tolist()

    
import cv2
import numpy as np
import imageio

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import scipy
from scipy import signal
import sys




def GenPairs(n):
    p = []
    n_p = int(np.floor(n/2))
    for i in range(0, n_p):
        p.append((2*i,2*i+1))
    if n%2==1:
        p.append((2*n_p-1,2*n_p))
    return p

def GenPairs(p_list):
    n=len(p_list)
    p = []
    n_p = int(np.floor(n/2))
    for i in range(0, n_p):
        p.append((p_list[2*i],p_list[2*i+1]))
    if n%2==1:
        p.append((p_list[2*n_p-1],p_list[2*n_p]))
    return p

def Video2Png(n):    
    level = 1
    img_list = list(range(1,n+1))
    #img_list = FrameSelect(img_list)
    if n<=1:
        print("Nothing to do.")
        return
    while(n>1):
        curr_pairs = GenPairs(img_list)
        img_list = StichPairs(curr_pairs,level)
        level = level+1
        if len(img_list)<n:
            n = len(img_list)
        else:
            print("No further reduction possible")
            break
        print(curr_pairs)
    
    
def StichPairs(pair_list, level):
    
    no_of_pairs = len(pair_list)
    successful_pairs = []
    counter = 0
    for i in range(0,no_of_pairs):
        # Read files
        fname1 = path + 'L'+str(level-1).zfill(2) + '_' + str(pair_list[i][0]).zfill(5) + '.jpg'
        fname2 = path + 'L'+str(level-1).zfill(2) + '_' + str(pair_list[i][1]).zfill(5) + '.jpg'
        
        ## AutoCrop to Eliminate White Spaces
        trainImg = imageio.imread(fname1)
        trainImg = AutoCrop(trainImg)
        trainImg_gray = cv2.cvtColor(trainImg, cv2.COLOR_RGB2GRAY)
        queryImg = imageio.imread(fname2)
        queryImg = AutoCrop(queryImg)
        queryImg_gray = cv2.cvtColor(queryImg, cv2.COLOR_RGB2GRAY)
        
        tres,s_ratio=Stich(trainImg_gray, queryImg_gray,trainImg,queryImg,feature_extractor)
        #tres1,s_ratio1=Stich(queryImg_gray,trainImg_gray,queryImg,trainImg,feature_extractor)

        if s_ratio<(1.1):
            res = tres
            #successful_pairs.append(i)
            cv2.imwrite(path + 'L'+str(level).zfill(2) + '_' + str(counter).zfill(5) + '.jpg', tres)
            successful_pairs.append(counter)
            counter = counter+1
        else:
            cv2.imwrite(path + 'L'+str(level).zfill(2) + '_' + str(counter).zfill(5) + '.jpg', imageio.imread(fname1))
            successful_pairs.append(counter)
            counter = counter+1
            cv2.imwrite(path + 'L'+str(level).zfill(2) + '_' + str(counter).zfill(5) + '.jpg', imageio.imread(fname2))
            successful_pairs.append(counter)
            counter = counter+1
            
    return successful_pairs

In [101]:
import numpy as np
def GenPairs(n):
    p = []
    n_p = int(np.floor(n/2))
    for i in range(0, n_p):
        p.append((2*i,2*i+1))
    if n%2==1:
        p.append((2*n_p-1,2*n_p))
    return p

def Hierarchical(n):
    if n<=1:
        return
    while(n>1):
        print("Calling: ",n)
        print(GenPairs(n))
        n = np.floor(n/2.0).astype('int')

In [72]:
%%time
import sys
import os
import glob

video_folder = '/home/medathati/Work/VideoStiching/VideoFrameSummary/Videos/21_April_Abhilash/'

for i in range(20,21):
    path = video_folder  + '_frames_noref'+ str(i).zfill(2) + '/'
    no_of_files = glob.glob(path + 'L00*.jpg')
    print("Processing video: ", i)
    print(path,'No of images: ',len(no_of_files))
    Video2Png(len(no_of_files))
    

#path = '/home/medathati/Work/VideoStiching/VideoFrameSummary/Videos/_frames2/'
#Video2Png(108)

Processing video:  20
/home/medathati/Work/VideoStiching/VideoFrameSummary/Videos/21_April_Abhilash/_frames_noref20/ No of images:  21
Raw matches (Brute force): 920
Shape of: (4, 1, 2)
Raw matches (Brute force): 771
Shape of: (4, 1, 2)
Raw matches (Brute force): 1569
Shape of: (4, 1, 2)
Raw matches (Brute force): 1950
Shape of: (4, 1, 2)
Raw matches (Brute force): 2415
Shape of: (4, 1, 2)
Raw matches (Brute force): 960
Shape of: (4, 1, 2)
Raw matches (Brute force): 1140
Shape of: (4, 1, 2)
Raw matches (Brute force): 955
Shape of: (4, 1, 2)
Raw matches (Brute force): 532
Shape of: (4, 1, 2)
Raw matches (Brute force): 1069
Shape of: (4, 1, 2)
Raw matches (Brute force): 1345
Shape of: (4, 1, 2)
[(1, 2), (3, 4), (5, 6), (7, 8), (9, 10), (11, 12), (13, 14), (15, 16), (17, 18), (19, 20), (20, 21)]
Raw matches (Brute force): 581
Shape of: (4, 1, 2)
Raw matches (Brute force): 1899
Shape of: (4, 1, 2)
Raw matches (Brute force): 1737
Shape of: (4, 1, 2)
Raw matches (Brute force): 722
Shape of: 

In [43]:
def FrameSelect(im_list):
    im_list = np.array(im_list)
    level = 1 
    no_of_images = len(im_list)
    effective_len = np.zeros(no_of_images)
    for i in range(0,no_of_images):
        # Read files
        fname1 = path + 'L'+str(level-1).zfill(2) + '_' + str(im_list[i]).zfill(5) + '.jpg'
        ## AutoCrop to Eliminate White Spaces
        trainImg = imageio.imread(fname1)
        effective_len[i] = AutoCropLen(trainImg)
    
    d_len = np.diff(effective_len)
    d_len[d_len>-20]=0
    s_im_list = im_list[np.nonzero(d_len)[0]]
    s_im_list = np.concatenate((s_im_list,[im_list[-1]]))
    return s_im_list.tolist()

im_list = list(range(1,180))
d_len = FrameSelect(im_list)

In [36]:
print(im_list[np.nonzero(d_len)[0]])

[  4   9  13  20  32  34  40  41  43  48  61  67  68  71  74  76 107 109
 110 111 118 139 140 141 156 166 167 174]


In [24]:
print(np.nonzero(d_len)[0])

[ 12  19  31  33  39  40  42  60  66  67  73  75 106 108 109 110 138 139
 140 155 165 166 173]


In [29]:
c = list(np.nonzero(d_len)[0])
im_list[c]

TypeError: list indices must be integers or slices, not list

In [39]:
np.concatenate((im_list,[10]))

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
       170, 171, 172, 173, 174, 175, 176, 177, 178, 179,  10])

In [70]:
%time
a = np.array([0,1,0,0,0,2,3,4,5])
b = np.nonzero(a)
print(a[b[0]])

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 4.77 µs
[1 2 3 4 5]


In [60]:
a = np.array([0,0,0])
print(len(np.nonzero(a)[0]))

0


In [94]:
def ExtractBackgroundFrame(im

SyntaxError: unexpected EOF while parsing (<ipython-input-94-d77627b182eb>, line 1)